In [1]:
%cd /Users/idawatibustan/Dev/shopee_data_science
%env PROJECT_PATH /Users/idawatibustan/Dev/shopee_data_science

/Users/idawatibustan/Dev/shopee_data_science
env: PROJECT_PATH=/Users/idawatibustan/Dev/shopee_data_science


In [2]:
from utils.envs import *
import pandas as pd
import numpy as np
import json
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
# from category_encoders import OrdinalEncoder
# from sklearn.model_selection import train_test_split
# import xgboost as xgb
# import lightgbm as lgb
# import string

In [3]:
beauty_train = pd.read_csv(beauty_train_repo)
beauty_val = pd.read_csv(beauty_val_repo)
beauty_train.head()

,itemid,title,image_path,Benefits,Brand,Colour_group,Product_texture,Skin_type
0,307504,nyx sex bomb pallete natural palette,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg,1.0,157.0,NaN,NaN,NaN
1,461203,etude house precious mineral any cushion pearl...,beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg,NaN,73.0,11.0,7.0,NaN
2,3592295,milani rose powder blush,beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg,NaN,393.0,20.0,6.0,NaN
3,4460167,etude house baby sweet sugar powder,beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg,NaN,73.0,NaN,6.0,NaN
4,5853995,bedak revlon color stay aqua mineral make up,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg,3.0,47.0,NaN,6.0,NaN


In [4]:
beauty_profile = json.load(open(beauty_profile_json))
beauty_profile.keys()

dict_keys(['Benefits', 'Brand', 'Colour_group', 'Product_texture', 'Skin_type'])

## Building Libraries of Secondary/ Tertiary Comparison 

In [5]:
""" get beauty features library (for convenience) """
print(beauty_train.columns)

benefits = beauty_profile['Benefits']
brands = beauty_profile['Brand']
colours = beauty_profile['Colour_group']
textures = beauty_profile['Product_texture']
skin_types = beauty_profile['Skin_type']

Index(['itemid', 'title', 'image_path', 'Benefits', 'Brand', 'Colour_group',
       'Product_texture', 'Skin_type'],
      dtype='object')


In [6]:
""" build secondary & tertiary beauty profile - brand """
brands_second = {}
brands_third = {}
temp = {}

""" if text contains dots, replace with spaces """
for b, val in brands.items():
    if "." in b:
        temp[b.replace(".", " ")] = val

""" append this to original library """
brands = {**brands, **temp}

for b, val in brands.items():
    """ if text contains spaces, combine them """
    num_spaces = len(b.split())
    for i in range(1, num_spaces):
        temp = b.replace(" ", '', i)
        brands_second[temp] = val

        """ if there are spaces, take each word """
        if len(temp.split())> 1:
            for text in temp.split():
                if len(text) > 2:
                    brands_third[text] = val

    """ if there are spaces, take each word"""
    if len(b.split()) > 1:
        for text in b.split():
            if len(text) > 2 and text != 'the' and text!= 'original':
                brands_third[text] = val

# output
# brands_second
# brands_third

In [7]:
""" fix known typo (manual) """
val = brands_second.get('natruerepublic')
brands_second['nature republic'] = val
brands_second['naturerepublic'] = val
val

383

In [8]:
""" store secondary / tertiary data """
data = {
    'Brands': brands_second,
}
with open('data/beauty_profile_secondary.json', 'w+') as file:
    json.dump(data, file)

data = {
    'Brands': brands_third,
}
with open('data/beauty_profile_tertiary.json', 'w+') as file:
    json.dump(data, file)


## For easy visualisation of data 

In [ ]:
""" get text details of all the features"""
for feature in beauty_profile.keys():
    key_to_text = {v: k for k, v in beauty_profile.get(feature).items()}
    beauty_train[feature+'_info'] = beauty_train[feature].map(key_to_text)


In [ ]:
beauty_train.drop(columns=['itemid', 'image_path']).head()

## heuristic functions

In [9]:
""" heuristic function :) """
def word_compare(title, library):
    texts = title.split()

    texts_lib = list(zip(texts, texts[1:]))
    texts_lib += list(zip(texts, texts[1:], texts[2:]))
    texts_lib += list(zip(texts, texts[1:], texts[2:], texts[3:]))

    texts_lib = texts + [' '.join(text) for text in texts_lib]
    
    for text in texts_lib:
        value = library.get(text, None)
        if value:
            return value
    return None

def get_brand(title):
    res = word_compare(title, brands)
    if res:
        return res
    res = word_compare(title, brands_second)
    if res:
        return res
    res = word_compare(title, brands_third)
    return res

def get_benefit(title):
    return word_compare(title, benefits)

def get_colour(title):
    return word_compare(title, colours)

def get_texture(title):
    return word_compare(title, textures)

def get_skin_type(title):
    return word_compare(title, skin_types)

heuristics = {
    'Benefits': get_benefit,
    'Brand': get_brand,
    'Colour_group': get_colour,
    'Product_texture': get_texture,
    'Skin_type': get_skin_type,
}

In [ ]:
# not used
def title_prediction(title):
    brand = None
    benefit = None
    colour = None
    texture = None
    skin_type = None
    
    texts = title.split()
    texts_lib = texts
    texts_lib += list(zip(texts, texts[1:]))
    texts_lib += list(zip(texts, texts[1:], texts[2:]))
    texts_lib += list(zip(texts, texts[1:], texts[2:], texts[3:]))

    for text in texts_lib:
        if not brand:
            brand = brands.get(text, None)
        if not benefit:
            benefit = benefits.get(text, None)
        if not colour:
            colour = colours.get(text, None)
        if not texture:
            texture = textures.get(text, None)
        if not skin_type:
            skin_type = skin_types.get(text, None)
        if brand and benefit and colour and texture and skin_type:
            break
    
    return {
        'brand': brand,
        'benefit': benefit,
        'colour': colour,
        'texture': texture,
        'skin_type': skin_type
    }

## Running heuristic functions 

In [10]:
""" run all heuristic functions """
from datetime import datetime
start_time = datetime.now()

for feature in beauty_profile.keys():
    beauty_train[feature+'_pred'] = beauty_train.title.map(heuristics[feature])

duration = datetime.now() - start_time
print(duration)

beauty_train.drop(columns=['image_path'])

0:00:16.228804


,itemid,title,Benefits,Brand,Colour_group,Product_texture,Skin_type,Benefits_pred,Brand_pred,Colour_group_pred,Product_texture_pred,Skin_type_pred
0,307504,nyx sex bomb pallete natural palette,1.0,157.0,NaN,NaN,NaN,1.0,157.0,NaN,NaN,NaN
1,461203,etude house precious mineral any cushion pearl...,NaN,73.0,11.0,7.0,NaN,NaN,73.0,NaN,7.0,NaN
2,3592295,milani rose powder blush,NaN,393.0,20.0,6.0,NaN,NaN,393.0,NaN,NaN,NaN
3,4460167,etude house baby sweet sugar powder,NaN,73.0,NaN,6.0,NaN,NaN,73.0,NaN,NaN,NaN
4,5853995,bedak revlon color stay aqua mineral make up,3.0,47.0,NaN,6.0,NaN,NaN,47.0,NaN,NaN,NaN
5,6208490,dr pure whitening cream,6.0,215.0,NaN,8.0,NaN,NaN,215.0,NaN,8.0,NaN
6,6959546,chanel powder blush malice,NaN,212.0,NaN,6.0,NaN,NaN,212.0,NaN,NaN,NaN
7,8984461,snail white cream original 100,NaN,NaN,22.0,8.0,0.0,NaN,283.0,NaN,8.0,NaN
8,9184082,sunprise all proof spf 50,6.0,NaN,NaN,NaN,NaN,6.0,62.0,NaN,NaN,NaN
9,10345338,eyebrow powder nyx satuan rp 15.000 pc,NaN,157.0,NaN,6.0,NaN,NaN,157.0,NaN,NaN,NaN


In [11]:
""" run all heuristic functions - prediction on separate df """
from datetime import datetime
start_time = datetime.now()

beauty_pred = beauty_train.filter(['title'])
for feature in beauty_profile.keys():
    beauty_pred[feature] = beauty_train.title.map(heuristics[feature])

duration = datetime.now() - start_time
print(duration)

beauty_pred

0:00:17.315034


,title,Benefits,Brand,Colour_group,Product_texture,Skin_type
0,nyx sex bomb pallete natural palette,1.0,157.0,NaN,NaN,NaN
1,etude house precious mineral any cushion pearl...,NaN,73.0,NaN,7.0,NaN
2,milani rose powder blush,NaN,393.0,NaN,NaN,NaN
3,etude house baby sweet sugar powder,NaN,73.0,NaN,NaN,NaN
4,bedak revlon color stay aqua mineral make up,NaN,47.0,NaN,NaN,NaN
5,dr pure whitening cream,NaN,215.0,NaN,8.0,NaN
6,chanel powder blush malice,NaN,212.0,NaN,NaN,NaN
7,snail white cream original 100,NaN,283.0,NaN,8.0,NaN
8,sunprise all proof spf 50,6.0,62.0,NaN,NaN,NaN
9,eyebrow powder nyx satuan rp 15.000 pc,NaN,157.0,NaN,NaN,NaN


In [ ]:
""" run single heuristic functions """
from datetime import datetime
start_time = datetime.now()

# beauty_train['Benefits_pred'] = beauty_train.title.map(get_benefit)
beauty_train['Brand_pred'] = beauty_train.title.map(get_brand)
# beauty_train['Colour_group_pred'] = beauty_train.title.map(get_colour)
# beauty_train['Product_texture_pred'] = beauty_train.title.map(get_texture)
# beauty_train['Skin_type_pred'] = beauty_train.title.map(get_skin_type)

duration = datetime.now() - start_time
print(duration)


## View result

In [12]:
""" view how many are NaN """
# for col in beauty_train.columns:
info_cols = [col for col in beauty_train.columns if "_info" in col]
beauty_train.drop(columns=info_cols).isna().sum()/len(beauty_train)


itemid                  0.000000
title                   0.000000
image_path              0.000000
Benefits                0.603759
Brand                   0.169078
Colour_group            0.576653
Product_texture         0.147559
Skin_type               0.796185
Benefits_pred           0.822463
Brand_pred              0.085822
Colour_group_pred       0.953256
Product_texture_pred    0.529459
Skin_type_pred          0.954837
dtype: float64

In [13]:
""" view how many are equal """
beauty_train.filter(beauty_pred.columns).eq(beauty_pred).sum()/len(beauty_pred)

title              1.000000
Benefits           0.174843
Brand              0.788986
Colour_group       0.046733
Product_texture    0.469735
Skin_type          0.045090
dtype: float64

In [14]:
""" view how many actually not NaN & correctly classified """
for feature in beauty_profile.keys():
    index = beauty_train.dropna(subset=[feature]).index
    correct_not_na = beauty_train.loc[index, feature].eq(beauty_pred.loc[index, feature]).sum()
    total_not_na = len(index)
    print(correct_not_na, correct_not_na/total_not_na, feature)

50107 0.44125365458452215 Benefits
226110 0.9495313444869986 Brand
13393 0.11039035969799875 Colour_group
134618 0.5510468900304959 Product_texture
12922 0.22122924156822463 Skin_type


## misc 

In [ ]:
print("incorrectly classified: ")
count = len(beauty_train.loc[(beauty_train.Brand != beauty_train.brand_pred), :].dropna(subset=['Brand']))
print(count, count/len(beauty_train))

In [ ]:
""" search in brand """
print([brand for brand in brands if "ha" in brand])